In [1]:
import torch
from llava.constants import X_TOKEN_INDEX, DEFAULT_X_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_X_token, get_model_name_from_path, KeywordsStoppingCriteria

/home/users/akane/miniconda3/envs/vllava/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/users/akane/miniconda3/envs/vllava/lib/python3.11/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/users/akane/miniconda3/envs/vllava/lib/python3.11/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/home/users/akane/miniconda3/envs/vllava/lib/python3.11/site-packages/torchvisi

In [27]:
disable_torch_init()
 
inp = 'Why is this video funny?'
model_path = 'LanguageBind/Video-LLaVA-7B'
device = 'cuda'
load_4bit, load_8bit = True, False
model_name = get_model_name_from_path(model_path)
tokenizer, model, processor, context_len = load_pretrained_model(model_path, None, model_name, load_8bit, load_4bit, device=device)
video_processor = processor['video']
conv_mode = "llava_v1"
conv = conv_templates[conv_mode].copy()
roles = conv.roles


Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.47s/it]
Some weights of the model checkpoint at LanguageBind/Video-LLaVA-7B were not used when initializing LlavaLlamaForCausalLM: ['model.video_tower.video_tower.encoder.layers.12.mlp.fc1.bias', 'model.video_tower.video_tower.encoder.layers.16.layer_norm1.bias', 'model.image_tower.image_tower.encoder.layers.23.layer_norm2.weight', 'model.video_tower.video_tower.encoder.layers.12.temporal_attn.k_proj.weight', 'model.image_tower.image_tower.encoder.layers.14.mlp.fc1.weight', 'model.video_tower.video_tower.encoder.layers.16.temporal_embedding', 'model.video_tower.video_tower.encoder.layers.17.layer_norm2.weight', 'model.video_tower.video_tower.encoder.layers.23.temporal_attn.q_proj.bias', 'model.image_tower.image_tower.encoder.layers.11.self_attn.out_proj.weight', 'model.video_tower.video_tower.encoder.layers.5.temporal_attn.out_proj.bias', 'model.video_tower.video_tower.encoder.layers.12.temporal_attn.q_proj.bias', 'model.


['Video', 'Image']


In [28]:
video = ['/workspace/akane/Video-LLaVA/llava/serve/examples/sample_demo_1.mp4'] * 1
video_tensor = video_processor(video, return_tensors='pt')['pixel_values']
if type(video_tensor) is list:
    tensor = [video.to(model.device, dtype=torch.float16) for video in video_tensor]
else:
    tensor = video_tensor.to(model.device, dtype=torch.float16)
key = ['video']

print(tensor.shape)


torch.Size([1, 3, 8, 224, 224])


In [29]:
print(f"{roles[1]}: {inp}")
inp = DEFAULT_X_TOKEN['VIDEO'] + '\n' + inp
conv.append_message(conv.roles[0], inp)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
print(f"{prompt=}")
input_ids = tokenizer_X_token(prompt, tokenizer, X_TOKEN_INDEX['VIDEO'], return_tensors='pt').unsqueeze(0).cuda()
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

input_ids.shape

ASSISTANT: Why is this video funny?
prompt="A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <video>\nWhy is this video funny? ASSISTANT:"


torch.Size([1, 50])

In [30]:
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=[tensor, key],
        do_sample=True,
        temperature=0.1,
        max_new_tokens=1024,
        use_cache=True,
        stopping_criteria=[stopping_criteria])

outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
print(outputs)

['video']
batch_idx=0
X_token_indices=tensor([35], device='cuda:0')


The video is funny because the baby is trying to read a book while sitting on the bed, but he is not able to read it properly. He is pointing to different parts of the book, and the video captures his attempts to read it.</s>
